# Common REST API Demo

[Common REST API Docs](https://enclave-markets.notion.site/Common-REST-API-9d546fa6282b4bad87ef43d189b9071b)

## Getting started

Note that for many of the endpoints such as deposit and withdrawal to work, you cannot use the sandbox environment.

### Auth

Before using, please read the [overview docs](https://enclave-markets.notion.site/REST-API-Overview-57966a627e5445bba573fd66475a553d)
as well as the [authentication docs](https://enclave-markets.notion.site/Rest-API-Authentication-3956dcfecbdc48269031cf052926c90d) 
to get an API key with View + Transfer to use the full demo, or just make a key with View to ensure no funds are moved.

### Prepare account

After following the instruction to get an API key and secret, for all the endpoints to work as shown, do the following:

1.  provision a deposit address for AVAX
2.  deposit (not from a smart wallet!) some AVAX to the address
3.  add a withdrawal address to your address book

Now you're all set :)


## Setup

Install dependencies. 

### Janky fix import

And add the enclave module's path to the system path so we can import the local module in jupyter. 
Alternatively in VSCode you can follow [this tutorial](https://stackoverflow.com/a/73954768),
or from the command line generally [this tutorial](https://stackoverflow.com/a/65182353).

If you follow the above, feel free to remove the next cell.

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Dependencies

Continue installing and importing dependencies normally.

In [2]:
%pip install requests

import enclave

Note: you may need to restart the kernel to use updated packages.


## Environment variables

Set the following variables for auth and to set the base url.

In [3]:
import enclave.models
from IPython.core.error import UsageError

# change to the env of your choice that matches the API keys: PROD, DEV, SANDBOX etc.
BASE_URL = enclave.models.DEV


### three ways to get API key and secret, tried in order.

# 1. as hardcoded strings
API_KEY: str = ""
API_SECRET: str = ""

# 2. as environment variables
if not(all([API_KEY, API_SECRET])):
    try:
        API_KEY = %env ENCLAVE_API_KEY
        API_SECRET = %env ENCLAVE_API_SECRET
    except UsageError:
        pass

# 3. from a .env file with `key` and `secret` set.
if not(all([API_KEY, API_SECRET])):
    %pip install python-dotenv
    import dotenv
    envs = dotenv.dotenv_values("dev.env")
    if envs and "key" in envs and "secret" in envs:
        API_KEY, API_SECRET = str(envs["key"]), str(envs["secret"])


if not(all([API_KEY, API_SECRET])):
    raise ValueError("Please provide API_KEY and API_SECRET")

Note: you may need to restart the kernel to use updated packages.


## Create client

Create a client with the variables and get authed hello.

If you get anything other than a 200 status code and the hello message with `"success": True` 
and your Enclave Account ID, then there is an issue -- <br>
either with auth or connection permissions to the URL.

In [4]:
from enclave.client import Client

client = Client(api_key=API_KEY, api_secret=API_SECRET, base_url=BASE_URL)

hello_res = client.authed_hello()
if 200 <= hello_res.status_code < 300:
    print(hello_json := hello_res.json())
    assert hello_json["success"] == True
else:
    print(f"Could not connect:\n{hello_res.text=}")

{'success': True, 'result': '5413750167799405746'}


## Common API

Demonstrate common API endpoints.

### Market info

In [5]:
markets = client.get_markets().json()
print('available markets')
import json
print(json.dumps(markets["result"], indent=2))

available markets
{
  "cross": {
    "tradingPairs": [
      {
        "market": "AVAX-USDC",
        "pair": {
          "base": "AVAX",
          "quote": "USDC"
        },
        "decimalPlaces": 6,
        "feeDecimal": 0.0004
      },
      {
        "market": "AAVE-USDC",
        "pair": {
          "base": "AAVE",
          "quote": "USDC"
        },
        "decimalPlaces": 6,
        "feeDecimal": 0.0005
      },
      {
        "market": "BTC.b-USDC",
        "pair": {
          "base": "BTC.b",
          "quote": "USDC"
        },
        "decimalPlaces": 6,
        "feeDecimal": 0.0005
      },
      {
        "market": "ETH-USDC",
        "pair": {
          "base": "ETH",
          "quote": "USDC"
        },
        "decimalPlaces": 6,
        "feeDecimal": 0.0005
      },
      {
        "market": "LINK-USDC",
        "pair": {
          "base": "LINK",
          "quote": "USDC"
        },
        "decimalPlaces": 6,
        "feeDecimal": 0.0005
      },
      {
       

### Wallet

Demonstrate wallet endpoints such as getting balances, deposit addresses, and withdrawal addresses.

In [6]:
balances = client.get_balances().json()
print('available balances')
print(json.dumps(balances["result"], indent=2))

available balances
[
  {
    "coin": "AAVE",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "AVAX",
    "total": "0.1",
    "reserved": "0",
    "free": "0.1",
    "usdValue": "2.67"
  },
  {
    "coin": "BTC",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "BTC.b",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "ETH",
    "total": "0.01",
    "reserved": "0",
    "free": "0.01",
    "usdValue": "23.54"
  },
  {
    "coin": "LINK",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "USDC",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "USDCa",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "USDT",
    "total": "0",
    "reserved": "0",
    "free": "0",
    "usdValue": "0"
  },
  {
    "coin": "

Per coin either manually or with get_balance

In [7]:
print(f'AVAX balance:\n{[x for x in balances["result"] if x["coin"] == "AVAX"]}')

balance_avax = client.get_balance("AVAX").json()
print(f'\n\nAVAX balance:\n{balance_avax["result"]=}')



AVAX balance:
[{'coin': 'AVAX', 'total': '0.1', 'reserved': '0', 'free': '0.1', 'usdValue': '2.67'}]


AVAX balance:
balance_avax["result"]={'accountId': '5413750167799405746', 'symbol': 'AVAX', 'totalBalance': '0.1', 'reservedBalance': '0', 'freeBalance': '0.1'}


Deposits and deposit addresses

In [8]:
deposit_addrs = client.get_deposit_addresses(["AVAX"]).json()
print(f"provisioned addresses:\n{json.dumps(deposit_addrs['result'], indent=2)}")

provisioned addresses:
[
  {
    "coin": "AVAX",
    "address": "70e4803cf4c2d9114d1549d784624dc3da27777f"
  },
  {
    "coin": "AVAX",
    "address": "39785f15a8efa39106c5dc15cefc33c9fe34a9e1"
  },
  {
    "coin": "AVAX",
    "address": "5cd6e5327fa01b589c7bc4df2e9f84b6ae10f6ee"
  }
]


Provision a new address and see it come up

In [9]:
provision_res = client.provision_address("AVAX").json()
print(f"provisioned address:\n{json.dumps(provision_res['result'], indent=2)}")

new_deposit_addrs = client.get_deposit_addresses(["AVAX"]).json()

# there is one more address now
assert(len(new_deposit_addrs["result"]) > len(deposit_addrs["result"]))

provisioned address:
{
  "accountId": "5413750167799405746",
  "symbol": "AVAX",
  "address": "0x2b9a4fac31f6a8746b9ff6abed28e0055536674b"
}


In [10]:
%pip install pandas -q
import pandas as pd
from io import StringIO

deposits_csv = client.get_deposits_csv().text
print(f"raw csv: {deposits_csv=}")

x = pd.read_csv(StringIO(deposits_csv))
display(x)

Note: you may need to restart the kernel to use updated packages.
raw csv: deposits_csv='id,time,coin,size,status\n0x143a0062630161fcb23aed7f54fc9ec4cc4a2546517638ba8c96af43882360f5,2023-12-07T18:47:07Z,ETH,0.01,confirmed\n0x89b62c6623bc21eb981f4fd6e4418693eda8774426533d75a56fa1cf51ed56d9,2023-12-07T17:51:03Z,AVAX,0.1,confirmed\n'


,id,time,coin,size,status
0,0x143a0062630161fcb23aed7f54fc9ec4cc4a25465176...,2023-12-07T18:47:07Z,ETH,0.01,confirmed
1,0x89b62c6623bc21eb981f4fd6e4418693eda877442653...,2023-12-07T17:51:03Z,AVAX,0.10,confirmed


Address book and withdrawals

Get one of the addresses and withdraw to it

In [11]:
# no current withdrawals
withdrawals = client.get_withdrawals().json()
print(f"withdrawals: {withdrawals=}")

withdrawals: withdrawals={'success': True, 'result': []}


Make sure you have added a withdrawal address to your address book on the web UI.

In [12]:
addr_book = client.get_address_book().json()
print(f"address book: {addr_book}")

withdrawal_addr = addr_book["result"]["addressBook"][0]["withdrawalAddress"]
print(f"\n a {withdrawal_addr=}")

address book: {'success': True, 'result': {'addressBook': [{'withdrawalAddress': '0x5A01855E59d847A12DaEa449166b0f97200e80f7', 'label': 'demo', 'lastUpdated': '2023-12-07T18:49:44.242394721Z'}]}}

 a withdrawal_addr='0x5A01855E59d847A12DaEa449166b0f97200e80f7'


Withdraw to it

In [13]:
import decimal
import time
WITHDRAW_AMOUNT = decimal.Decimal(0.01)


withdrawal_res = client.withdraw(withdrawal_addr, WITHDRAW_AMOUNT, str(int(time.time())), "AVAX").json()
print(f"withdrawal: {withdrawal_res}")

withdrawal_id = withdrawal_res["result"]["withdrawal_id"]
customer_withdrawal_id = withdrawal_res["result"]["customer_withdrawal_id"]

withdrawal: {'success': True, 'result': {'withdrawal_status': 'WITHDRAWAL_PENDING', 'withdrawal_id': 'b9b850deec0be2f5f20ce76cb3b589c3e0aa40b4f8b305931d7f4bf886806e87', 'customer_withdrawal_id': '1701974995'}}


In [14]:
# there is a withdrawal now
withdrawals = client.get_withdrawals().json()
print(f"withdrawals: {withdrawals=}")

# get by txid
txid_withdrawal = withdrawals["result"][0]["txid"]
withdrawal_res = client.get_withdrawal_by_txid(txid_withdrawal).json()
print(f"\nwithdrawal by txid: {withdrawal_res}")

# get by internal id
withdrawal_res_by_id = client.get_withdrawal(internal_id=withdrawal_id).json()
print(f"\nwithdrawal by id: {withdrawal_res_by_id}")

# get by customer id
withdrawal_res_by_cid = client.get_withdrawal(custom_id=customer_withdrawal_id).json()
print(f"\nwithdrawal by customer id: {withdrawal_res_by_cid}")



withdrawals: withdrawals={'success': True, 'result': [{'coin': 'AVAX', 'size': '0.01000000000000000020816681711721685132943093776702880859375', 'status': 'WITHDRAWAL_PENDING', 'address': '0x5A01855E59d847A12DaEa449166b0f97200e80f7', 'withdrawal_id': 'b9b850deec0be2f5f20ce76cb3b589c3e0aa40b4f8b305931d7f4bf886806e87', 'txid': '', 'customer_withdrawal_id': '1701974995', 'time': '2023-12-07T18:49:55.786267895Z'}]}

withdrawal by txid: {'success': True, 'result': [{'coin': 'AVAX', 'size': '0.01000000000000000020816681711721685132943093776702880859375', 'status': 'WITHDRAWAL_PENDING', 'address': '0x5A01855E59d847A12DaEa449166b0f97200e80f7', 'withdrawal_id': 'b9b850deec0be2f5f20ce76cb3b589c3e0aa40b4f8b305931d7f4bf886806e87', 'txid': '', 'customer_withdrawal_id': '1701974995', 'time': '2023-12-07T18:49:55.786267895Z'}]}

withdrawal by id: {'success': True, 'result': {'original_request': {'account_id': '5413750167799405746', 'customer_withdrawal_id': '1701974995', 'symbol': 'AVAX', 'amount': '0